## [Day 15](https://adventofcode.com/2021/day/15): Chiton
### Part I
- **Unknown**: The lowest risk of a path from the top left to the bottom right.
- **Data**: A map of risk level.
- **Condition**:
  - Add up the risk levels of each position you enter.
  - You cannot move diagonally.
  - We don't need to store the shortest path.

In [1]:
from pathlib import Path
import numpy as np
import heapq

In [2]:
def parse(input):
    """Return a map, start and end positions."""
    maze = Path('data/'+input).read_text().strip()
    maze = np.array([list(line) for line in maze.split('\n')]).astype(int)

    return maze, (0,0), tuple(np.array(maze.shape)-1)

maze, start, end = parse('AoC2021_15ex.txt')
maze.shape, start, end

((10, 10), (0, 0), (9, 9))

In [3]:
def get_neighbors(maze, current):
    """Find all the neighbours of a position"""

    x, y = current
    for dx, dy in [(-1,0), (1,0), (0,1), (0,-1)]:
        if not(0 <= x+dx < maze.shape[0] and 0 <= y+dy < maze.shape[1]):
            continue
        yield (x+dx, y+dy)

for pos in [start, (1,1), end]:
    print(list(get_neighbors(maze, pos)))

[(1, 0), (0, 1)]
[(0, 1), (2, 1), (1, 2), (1, 0)]
[(8, 9), (9, 8)]


Three data structures are kept and updated throughout the Dijkstra algoritm:
- `tentative` is a dict of tentative risk levels in a form {`pos` : risk} and starts with `{start : 0}` because the risk level of starting position is not counted.
- `certain` is set of positions for which the risk level in `tentative` is certain to be the lowest. It starts empty.
- `candidates` is a heap of positions that have calculated risk in a form of a tuple `(risk, position)` and starts with `[(0, start)]`.

In [4]:
def find_safer_path(maze, tentative, positions, through):
    """Find a path with lower risk"""
    for position in positions:
        if position in tentative and tentative[position] <= tentative[through] + maze[position]:
            continue
        yield position, tentative[through] + maze[position]

def calculate_lowest_risk(input):
    """Find the path with a lowest risk and return that risk."""

    maze, start, end = parse(input)
    tentative = {start : 0}
    candidates = [(0, start)]
    certain = set()
    while end not in certain and len(candidates) > 0:
        _ignored, current = heapq.heappop(candidates)
        if current in certain:
            continue
        certain.add(current)
        neighbours = set(get_neighbors(maze, current)) - certain
        safer = find_safer_path(maze, tentative, neighbours, current)
        for neighbour, risk in safer:
            tentative[neighbour] = risk
            heapq.heappush(candidates, (risk, neighbour))
    if end in tentative:
        return tentative[end]
    else:
        print('Something got wrong')
        return -1

In [5]:
for input in ['AoC2021_15ex.txt', 'AoC2021_15.txt']:
    print(input, ':', calculate_lowest_risk(input))

AoC2021_15ex.txt : 40
AoC2021_15.txt : 398


### Part II
- **Unknown**: The lowest total risk of any path from the top left to the bottom right on the full map.
- **Data**: The 5x5 area built from original map by incrementing and rolling risk levels.
- **Condition**:
  - Each time the tile (original map) repeats to the right or downward, all of its risk levels are 1 higher than the tile immediately up or left of it.
  - The risk levels above `9` wrap back around to `1`.
- **Plan**:
  - Create a function for incrementing a matrix.
  - Construct the full area model.
  - Find the lowest risk path and calculate its risk.


In [6]:
maze, start, end = parse('AoC2021_15ex.txt')
maze

array([[1, 1, 6, 3, 7, 5, 1, 7, 4, 2],
       [1, 3, 8, 1, 3, 7, 3, 6, 7, 2],
       [2, 1, 3, 6, 5, 1, 1, 3, 2, 8],
       [3, 6, 9, 4, 9, 3, 1, 5, 6, 9],
       [7, 4, 6, 3, 4, 1, 7, 1, 1, 1],
       [1, 3, 1, 9, 1, 2, 8, 1, 3, 7],
       [1, 3, 5, 9, 9, 1, 2, 4, 2, 1],
       [3, 1, 2, 5, 4, 2, 1, 6, 3, 9],
       [1, 2, 9, 3, 1, 3, 8, 5, 2, 1],
       [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]])

In [10]:
def increment_matrix(mat):
    """Increments matrix and wraps 10 -> 1"""
    new = mat.copy() + 1
    return np.where(new < 10, new, 1)

increment_matrix(maze)

array([[2, 2, 7, 4, 8, 6, 2, 8, 5, 3],
       [2, 4, 9, 2, 4, 8, 4, 7, 8, 3],
       [3, 2, 4, 7, 6, 2, 2, 4, 3, 9],
       [4, 7, 1, 5, 1, 4, 2, 6, 7, 1],
       [8, 5, 7, 4, 5, 2, 8, 2, 2, 2],
       [2, 4, 2, 1, 2, 3, 9, 2, 4, 8],
       [2, 4, 6, 1, 1, 2, 3, 5, 3, 2],
       [4, 2, 3, 6, 5, 3, 2, 7, 4, 1],
       [2, 3, 1, 4, 2, 4, 9, 6, 3, 2],
       [3, 4, 2, 2, 1, 5, 5, 6, 9, 2]])

In [15]:
def build_full_maze(maze):
    """Build the full maze by stacking incremented tiles."""
    full = maze.copy()
    
    # Stack tile horiontally
    inc = maze.copy()
    for _ in range(4):
        inc = increment_matrix(inc)
        full = np.hstack((full,inc))
    
    # Stack a row of tiles vertically
    inc = full.copy()
    for _ in range(4):
        inc = increment_matrix(inc)
        full = np.vstack((full,inc))

    return full

full = build_full_maze(maze)
full.shape

(50, 50)

In [17]:
def solve_part2(input):
    """Find the path with a lowest risk in a full map and return that risk."""

    maze, start, end = parse(input)
    
    maze = build_full_maze(maze)
    end = tuple(np.array(maze.shape)-1)

    tentative = {start : 0}
    candidates = [(0, start)]
    certain = set()
    while end not in certain and len(candidates) > 0:
        _ignored, current = heapq.heappop(candidates)
        if current in certain:
            continue
        certain.add(current)
        neighbours = set(get_neighbors(maze, current)) - certain
        safer = find_safer_path(maze, tentative, neighbours, current)
        for neighbour, risk in safer:
            tentative[neighbour] = risk
            heapq.heappush(candidates, (risk, neighbour))
    if end in tentative:
        return tentative[end]
    else:
        print('Something got wrong')
        return -1

for input in ['AoC2021_15ex.txt', 'AoC2021_15.txt']:
    print(input, ':', solve_part2(input))

AoC2021_15ex.txt : 315
AoC2021_15.txt : 2817
